In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import math

# Dataprep เตรียมข้อมูล

In [246]:
data = pd.read_csv('./Salary.csv')
#อ่านไฟล์ csv ด้วย pandas แล้ว assign เข้าตัวแปร data

In [247]:
# อ่านข้อมูลดู
data.head(5)

In [248]:
# สรุปภาพรวมของข้อมูล
data.describe()

In [249]:
from sklearn.model_selection import train_test_split

In [250]:
x_train, x_test, y_train, y_test = train_test_split(data['Salary'], data['YearsExperience'], random_state=69420)

In [251]:
plt.plot(data['Salary']) # เงินเดือน

In [252]:
plt.plot(data['YearsExperience']) # ประสบการณ์ทำงาน

# สร้าง Model Linear Regression Modelling

In [253]:
class ln():
    # setup parameters
    def __init__(self):
        self.axis = np.random.normal(-1,1,(1,1))
        self.intercept = np.random.normal(-1,1,(1,1))
        print('Axis:', self.axis, 'Intercept:', self.intercept)
        
    # normalizer scaling down by 1000 denominators to make compuational more effective
    def norm(self,x, reverse=False):
        scale = 1000
        if not reverse:
            x = x/scale
            return x
        elif reverse:
            x = x*scale
            return x
    
    # function just to change weight (make code cleaner)
    def update_weight(self, weight, intercept):
        self.axis = weight
        self.intercept = intercept
        
    # plotting the equation line
    def plot(self, data):
        # [0] to get data out of list (plt.plot require 1 dimensional data)
        m = self.axis[0]
        b = self.intercept[0]
        print(f'm{m} b{b}')
        x = np.array(data[data.columns[0]])
        y = np.array(data[data.columns[1]])
        y_pred = self.forward(x).tolist()[0]
        y_pred = [self.norm(x, reverse=True) for x in y_pred]
        plt.plot(x, y, 'o')
        
        plt.plot(x, y_pred)
        plt.show()
    
    # Cost Function
    def loss(self, real, pred):
        loss = np.sqrt((np.sum((real-pred)**2))/real.shape[1])
        return loss
    
    # Find Gradient
    def grad(self, real, pred, x):
        # partial derivative of axis & intercept
        grad_axis = 0
        grad_intercept = 0
        n = len(x)
        # Sum of
        for i in range(n):

            # Partial Derivatives
            grad_axis += (2/n) * x[i] * (real[0,i]-pred[i])
            grad_intercept += (2/n) * (real[0,i] -pred[i])
        return grad_axis, grad_intercept
    
    # Use To Predict
    def forward(self, x):
        x = self.norm(x)
        y = (self.axis*np.array(x)+self.intercept)
        return y
    
    # Training model
    def fit(self, x, y, epoch=1, lr=0.001):
        # Training Loop
        for i in range(epoch):
            real = self.norm(y)
            pred = self.forward(x)
            # compute loss(for logging)
            loss = self.loss(pred, real)
            if i % 100 == 0:
                print('MSE:', loss)
            
            # calc grad
            grad_axis, grad_intercept = self.grad(pred, real, x)
            # take a descent step
            new_axis = self.axis - (lr*grad_axis)
            new_intercept = self.intercept - (lr*grad_intercept)

            # update weight
            self.update_weight(new_axis, new_intercept)

In [254]:
model = ln()

In [255]:
model.plot(data)

In [256]:
model.fit(np.array(data['YearsExperience']), np.array(data['Salary']), epoch=1000, lr=0.1)
# model.fit(x_train, y_train, epoch=10, lr=0.001)

In [257]:
model.plot(data)

In [258]:
pred = model.forward(data['YearsExperience'])

In [259]:
predt = pred.tolist()[0]
predt = [model.norm(x, reverse=True) for x in predt]
real_pred = pd.DataFrame()
real_pred['real'] = data['Salary']
real_pred['pred'] = predt
real_pred['yearx'] = data['YearsExperience']
real_pred['pred_to_real_ratio'] = real_pred['pred']/(real_pred['real']/100)

In [260]:
real_pred

In [261]:
real_pred['pred_to_real_ratio'].mean()

In [262]:
print(model.axis, model.intercept)